In [120]:
from bs4 import BeautifulSoup
import requests
import datetime
from dateutil.relativedelta import relativedelta
import re
from tqdm import tqdm
import pandas as pd

In [72]:
UID_PINTUAIR=140
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2024, 3, 1)

#divide into months so scrape will work
selected_months=[]
current_date = start_date
while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    current_date += relativedelta(months=1)
    selected_months.append([formatted_date, (current_date-relativedelta(days=1)).strftime("%Y-%m-%d")])
selected_months[-1][-1]="2024-03-01"
selected_months

[['2023-01-01', '2023-01-31'],
 ['2023-02-01', '2023-02-28'],
 ['2023-03-01', '2023-03-31'],
 ['2023-04-01', '2023-04-30'],
 ['2023-05-01', '2023-05-31'],
 ['2023-06-01', '2023-06-30'],
 ['2023-07-01', '2023-07-31'],
 ['2023-08-01', '2023-08-31'],
 ['2023-09-01', '2023-09-30'],
 ['2023-10-01', '2023-10-31'],
 ['2023-11-01', '2023-11-30'],
 ['2023-12-01', '2023-12-31'],
 ['2024-01-01', '2024-01-31'],
 ['2024-02-01', '2024-02-29'],
 ['2024-03-01', '2024-03-01']]

In [122]:
thedata=[]
for i in tqdm(selected_months):
    URL = f'https://ews.tri3a.com/Pages/GenerateDataTinggiAir.aspx?IdPintuAir={UID_PINTUAIR}&StartDate={i[0]}&EndDate={i[1]}'
    r = requests.get(URL)
    soup=BeautifulSoup(r.content)
    temp=re.split(r';', soup.find("p").text)
    for j in temp:
        try:
            if i[0] <= re.search(r'\d{4}-\d{2}-\d{2}', j)[0] <=i[1]:
                thedata.append(j.split(","))
        except:
            pass

100%|██████████| 15/15 [00:20<00:00,  1.35s/it]


In [124]:
pd.DataFrame(thedata, columns=["date", "Sea level (cm)"]).to_csv("../Data/Raw/Sea level.csv", index=False)